In [ ]:
import degirum as dg
import requests
import json
from dg_coco_utils import *
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [ ]:
config=open('./coco_configs/Efficientdet_lite1_coco_config.json')
config_options=json.load(config)
config.close()

ai_server_address=''
models=dg.connect_model_zoo(ai_server_address)
ground_truth_annotations_path=" "
image_folder_path=" "
pred_json='yolov5m_orca_predictions.json'
print_frequency=50

In [ ]:
model_name=config_options['MODEL_PARAMETERS'][0]['ModelName']
eval_model=models.load_model(model_name)
eval_model.output_confidence_threshold=config_options['POST_PROCESS'][0]['OutputConfThreshold']
eval_model.output_nms_threshold=config_options['POST_PROCESS'][0]['OutputNMSThreshold']
eval_model.output_max_detections=config_options['POST_PROCESS'][0]['MaxDetections']
eval_model.output_max_detections_per_class=config_options['POST_PROCESS'][0]['MaxDetectionsPerClass']
eval_model.output_max_classes_per_detection=config_options['POST_PROCESS'][0]['MaxClassesPerDetection']
eval_model.output_use_regular_nms=config_options['POST_PROCESS'][0]['UseRegularNMS']
eval_model.input_resize_method=config_options['PRE_PROCESS'][0]['InputResizeMethod']
eval_model.input_pad_method=config_options['PRE_PROCESS'][0]['InputPadMethod']
eval_model.image_backend=config_options['PRE_PROCESS'][0]['ImageBackend']
class_map=config_options['POST_PROCESS'][0]['ClassMap']


In [ ]:
jdict=[]
anno = COCO(ground_truth_json)
num_images=len(anno.dataset['images'])
files_dict=anno.dataset['images'][0:num_images]

for image_number in range(0,num_images):
    if image_number%print_frequency==print_frequency-1:
        print(image_number+1)
    image_id=files_dict[image_number]['id']
    path=image_folder_path + files_dict[image_number]['file_name']
    predictions=eval_model(path)
    save_results_coco_json(predictions.results,jdict,image_id,class_map)
with open(pred_json, 'w') as f:
    json.dump(jdict, f)

pred = anno.loadRes(pred_json)
eval_obj = COCOeval(anno, pred, 'bbox')
eval_obj.params.imgIds = [file['id'] for file in files_dict]  # image IDs to evaluate
eval_obj.evaluate()
eval_obj.accumulate()
eval_obj.summarize()
map_all, map50 = eval_obj.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)